<a href="https://colab.research.google.com/github/agbaezehenry/QuantitativeFinance/blob/main/QuantFinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoo_fin

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.figure_factory as ff
import scipy.stats as stats

# Set parameters
sigma = 0.3  # Annualized volatility
mu = 0.1  # Annualized return
dt = 1/252  # One trading day
Nt = 252  # Number of trading days in a year
Np = 10000  # Number of simulations

# Generate standard normal random draws
np.random.seed(0)
z = np.random.normal(size=(Nt, Np))

# Scale and sum to generate log returns
r = mu * dt + z * sigma * np.sqrt(dt)
s = np.cumsum(r, axis=0)

# Convert log returns to prices
P = np.exp(s)

# Plot simulated price paths
fig = go.Figure(data=[go.Scatter(y=P[:, i]) for i in range(3)])
fig.update_layout(title='Simulated Price Paths', xaxis_title='Time', yaxis_title='Price')
fig.show()

# Calculate simple returns from the terminal price
R = P[-1, :] - 1

# Plot histogram of returns
fig = ff.create_distplot([R], ['Returns'], bin_size=.005)
fig.update_layout(title='Histogram of Returns', xaxis_title='Return', yaxis_title='Frequency')
fig.show()

# Plot histogram of log(1+R)
fig = ff.create_distplot([np.log1p(R)], ['log(1+Returns)'], bin_size=.005)
fig.update_layout(title='Histogram of log(1+Returns)', xaxis_title='log(1+Return)', yaxis_title='Frequency')
fig.show()

# Plot empirical cumulative distribution function (ECDF)
df = pd.DataFrame(R, columns=['Return'])
df = df.sort_values(by='Return')
df['ECDF'] = np.arange(1, len(df)+1) / len(df)
fig = go.Figure(data=go.Scatter(x=df['Return'], y=df['ECDF'], mode='lines'))
fig.update_layout(title='ECDF of Returns', xaxis_title='Return', yaxis_title='ECDF')
fig.show()

# Plot sorted returns
sorted_R = np.sort(R)
fig = go.Figure(data=go.Bar(x=np.arange(len(sorted_R)), y=sorted_R))
fig.update_layout(title='Sorted Returns', xaxis_title='Index', yaxis_title='Return')
fig.show()

# Q-Q plot
# qq = stats.probplot(R, dist="norm", plot=None)
# fig = go.Figure(data=go.Scatter(x=qq[0][0], y=qq[0][1], mode='markers'))
# fig.add_trace(go.Scatter(x=qq[0][0], y=stats.norm.pdf(qq[0][0], *qq[1]), mode='lines'))
# fig.update_layout(title='Q-Q plot of Returns', xaxis_title='Theoretical Quantiles', yaxis_title='Sample Quantiles')
# fig.show()

# Testing Random walks


In [1]:

import pandas as pd
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.stats import norm
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# ================================Set Parameters ===============================

symbol = "TR"
company_name = "Tootsie Roll"  #future def get_company_name(symbol): ->
start_date = "2010-12-31"
end_date = "2017-12-31"
# ================================================================================

def get_stock_prices(company_ticker="AAPL", start_date="2010-12-31", end_date="2017-12-31"):
    """
    Retrieves the stock prices for a given company within a specified date range.

    Args:
        company_ticker (str): Ticker symbol of the company (default: "AAPL" for Apple Inc.).
        start_date (str): Start date in the format "YYYY-MM-DD" (default: "2010-12-31").
        end_date (str): End date in the format "YYYY-MM-DD" (default: "2017-12-31").

    Returns:
        pandas.DataFrame: DataFrame with the open and close values indexed by time.
    """
    stock = yf.download(company_ticker)  # Download the entire available data
    if stock.empty:
        raise ValueError(f"No data available for the ticker symbol: {company_ticker}")

    stock_data = stock.loc[:, ['Open', 'Close', 'Low', 'Adj Close', 'Volume', 'High']]  # Extract desired columns
    stock_data.index.name = 'Date'  # Set the index name to 'Date'

    if start_date or end_date:
        stock_data = stock_data.loc[start_date:end_date]  # Filter the stock data based on the specified date range

    stock_data['log_return'] = np.log(stock_data['Adj Close']).diff()
    stock_data['cum_r'] = stock_data['log_return'].cumsum()
    stock_data['log_return_percentage'] = np.log(stock_data['Adj Close']).diff() * 100

    return stock_data.dropna().reset_index()



# Variance Ratio Test
def variance_c(X, q):
    T = len(X) - 1
    mu = (X[T] - X[0]) / T
    m = (T - q) * (T - q + 1) * q / T
    sumsq = 0
    for t in range(q, T+1):
        sumsq += (X[t] - X[t-q] - q * mu) ** 2
    return sumsq / m

def z(X, q):
    T = len(X) - 1
    c = np.sqrt(T * (3 * q) / (2 * (2 * q - 1) * (q - 1)))
    M = variance_c(X, q) / variance_c(X, 1) - 1
    z = c * M
    return z


df = get_stock_prices(company_ticker= symbol, start_date = start_date , end_date = end_date)


# Summary statistics
mean_return = np.mean(df['log_return']) * 252
volatility = np.std(df['log_return']) * np.sqrt(252)



# Variance plot
Variance = np.var(np.diff(np.log(df['Adj Close'])))
for n in range(2, 101):
    Variance = np.append(Variance, np.var(np.diff(np.log(df['Adj Close'][np.arange(n-1, len(df['Adj Close']), n)]))))



Vc = np.zeros(100)
for q in range(1, 101):
    Vc[q-1] = variance_c(np.log(df['Adj Close']), q)

z_stats = np.zeros(100)
for q in range(2, 101):
    z_stats[q-1] = z(np.log(df['Adj Close']), q)

p_values = 2 * (1 - norm.cdf(np.abs(z_stats)))



# Scaling of volatility
N = len(df['log_return'])
P = df['Adj Close'
]
sigma = np.zeros(100)
for n in range(2, 101):
    sigma[n-1] = np.sqrt(252/n) * np.std(np.diff(np.log(P[np.arange(n-1, len(P), n)])))


# Scaling of volatility simulation
N = 1000  # Number of simulation trials
P_MC = np.exp(np.cumsum(np.random.randn(N) * 0.02))
sigma_MC = np.zeros(100)
for n in range(2, 101):
    sigma_MC[n-1] = np.sqrt(252/n) * np.std(np.diff(np.log(P_MC[np.arange(n-1, N, n)])))


# =======================================================Plots==================================================


fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{"colspan": 2}, None], [{}, {}]],
    subplot_titles=(
        company_name + " Adjusted Price " + start_date + "-" + end_date,
        company_name + " Daily Returns " + start_date + "-" + end_date,
        "White Noise Process with " + company_name + " Volatility"
    )
)
# Plotting TR Adjusted Price 1988-2017 (spanning top two grids)
fig.add_trace(go.Scatter(x=df['Date'], y=df['Adj Close'], mode='lines'), row=1, col=1)

# Plotting TR Daily Returns 1988-2017
fig.add_trace(go.Scatter(x=df['Date'], y=df['log_return'], mode='lines'), row=2, col=1)

# Plotting White Noise Process with TR Volatility
fig.add_trace(go.Scatter(x=df['Date'], y=np.random.randn(len(df)) * np.std(df['log_return']), mode='lines'), row=2, col=2)

fig.update_layout(showlegend=False, title_text= company_name)
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_xaxes(title_text="Time", row=2, col=1)
fig.update_yaxes(title_text="Price", row=2, col=1)
fig.update_xaxes(title_text="Time", row=2, col=2)
fig.update_yaxes(title_text="Price", row=2, col=2)

# Adjust the subplot spacing
fig.update_layout(height=600, width=1000, showlegend=False, autosize=False, margin=dict(l=50, r=50, t=100, b=50),
                  paper_bgcolor='white')

fig.show()

# Create a new figure with two columns and three rows
fig = make_subplots(rows=3, cols=2, subplot_titles=("Histogram of Returns",
                                                    "Variance of Returns From n-day Observations",
                                                    "z Statistics of Variance Ratio Test",
                                                    "Volatility Scaling of Returns From n-day Observations ( " + company_name + " )",
                                                    "Volatility Scaling of Returns From n-day Observations (Sim)"))

# Histogram of returns
fig.add_trace(go.Histogram(x=df['log_return'], nbinsx=50), row=1, col=1)
fig.update_layout(xaxis_title='Returns', yaxis_title='Frequency', title='Histogram of Returns')

# Variance plot
Variance = np.var(np.diff(np.log(df['Adj Close'])))
for n in range(2, 101):
    Variance = np.append(Variance, np.var(np.diff(np.log(df['Adj Close'][np.arange(n-1, len(df['Adj Close']), n)]))))
fig.add_trace(go.Scatter(x=np.arange(1, 101), y=Variance, mode='lines'), row=1, col=2)
fig.update_layout(xaxis_title='n', yaxis_title='Variance', title='Variance of Returns From n-day Observations')

# z Statistics of Variance Ratio Test
fig.add_trace(go.Bar(x=np.arange(2, 101), y=z_stats), row=2, col=1)
fig.update_layout(xaxis_title='q', yaxis_title='z', title='z Statistics of Variance Ratio Test')

# Volatility Scaling of Returns From n-day Observations (TR)
fig.add_trace(go.Bar(x=np.arange(2, 101), y=sigma), row=2, col=2)
fig.update_layout(xaxis_title='n', yaxis_title='Standard Deviation (annualized) / sqrt(n)',
                   title='Volatility Scaling of Returns From n-day Observations (TR)')

# Volatility Scaling of Returns From n-day Observations (Sim)
fig.add_trace(go.Bar(x=np.arange(2, 101), y=sigma_MC), row=3, col=1)
fig.update_layout(xaxis_title='n', yaxis_title='Standard Deviation (annualized) / sqrt(n)',
                   title='Volatility Scaling of Returns From n-day Observations (Sim)')

# Adjust the subplot spacing
fig.update_layout(height=600, width=1000, showlegend=False, autosize=False, margin=dict(l=50, r=50, t=100, b=50),
                  paper_bgcolor='white')


fig.show()
print(f"Annualized mean return: {mean_return:.4f}")
print(f"Annualized volatility: {volatility:.4f}")

print(df.describe())
# ======================================================================================================================


[*********************100%***********************]  1 of 1 completed


Annualized mean return: 0.0675
Annualized volatility: 0.2036
              Open        Close          Low    Adj Close        Volume  \
count  1761.000000  1761.000000  1761.000000  1761.000000  1.761000e+03   
mean     25.324984    25.327149    25.088840    23.021884  1.122764e+05   
std       4.947189     4.943446     4.898524     5.079396  7.768639e+04   
min      16.738541    16.631243    16.577595    14.486123  7.437000e+03   
25%      21.605288    21.597624    21.452003    18.769209  7.104600e+04   
50%      24.803238    24.803238    24.511156    22.404253  9.399400e+04   
75%      29.708246    29.639238    29.406334    27.517485  1.281000e+05   
max      35.539482    35.841396    35.151306    33.487690  1.040035e+06   

              High   log_return        cum_r  log_return_percentage  
count  1761.000000  1761.000000  1761.000000            1761.000000  
mean     25.550505     0.000268     0.165300               0.026781  
std       4.987479     0.012830     0.223402         